#Limpeza

## Carregar Bases

In [ ]:
import pickle
import pandas as pd

In [ ]:
with open('/content/drive/MyDrive/TCC/Bases/mda_2020.pkl', 'rb') as f:  # jupyter notebook saved
    df = pickle.load(f)
f.close()

df['sentimento']=""
df.to_pickle('/content/drive/MyDrive/TCC/Bases_pt_2/mda_2020.pkl')

## Limpeza

In [ ]:
import pickle
import pandas as pd
with open('/content/drive/MyDrive/TCC/Bases_pt_2/mda_2016.pkl', 'rb') as f:  # jupyter notebook saved
    df = pickle.load(f)
f.close()

In [ ]:
df_empresa = pd.DataFrame()
df_empresa['mda'] = df['mda_2020_1']
df_empresa['Symbol']= df['Symbol']
df=[]

In [ ]:
dicionario = {'ITEM 7':'','MILLIONS':'','BILLIONS':'','MILLION':'','BILLION':'','THOUSANDS':'','THOUSAND':'','MANAGEMENT’S DISCUSSION AND ANALYSIS OF FINANCIAL CONDITION AND RESULTS OF OPERATIONS':''}
df_empresa['mda']= df_empresa.replace({'mda': dicionario},regex=True)
df_empresa['mda'][0]

'. MD&A.\n\nTHE COMPANIES’ RESULTS OF OPERATIONS CAN BE AFFECTED BY CHANGES IN THE WEATHER.\n\nFLUCTUATIONS IN WEATHER CAN AFFECT DEMAND FOR THE COMPANIES’ SERVICES. FOR EXAMPLE, MILDER THAN NORMAL WEATHER CAN REDUCE DEMAND FOR ELECTRICITY AND GAS TRANS\n\nMISSION AND DISTRIBUTION SERVICES. IN ADDITION, SEVERE WEATHER, INCLUDING\n\nHURRICANES, WINTER STORMS, EARTHQUAKES, FLOODS AND OTHER NATURAL DISASTERS CAN DISRUPT OPERATION OF THE COMPANIES’ FACILITIES AND CAUSE SERVICE OUTAGES, PRODUCTION DELAYS AND PROPERTY DAMAGE THAT REQUIRE INCURRING ADDITIONAL EXPENSES. CHANGES\n\nIN WEATHER CONDITIONS CAN RESULT IN REDUCED WATER LEVELS OR CHANGES IN WATER TEMPERATURES THAT COULD ADVERSELY AFFECT OPERATIONS AT SOME OF THE COMPANIES’ POWER STATIONS. FURTHERMORE, THE COMPANIES’ OPERATIONS COULD BE ADVERSELY AFFECTED\n\nAND THEIR PHYSICAL PLANT PLACED AT GREATER RISK OF DAMAGE SHOULD CHANGES IN GLOBAL CLIMATE PRODUCE, AMONG OTHER POSSIBLE CONDITIONS, UNUSUAL VARIATIONS IN TEMPERATURE AND WEATHER 

In [ ]:
# Remover caracteres da letra
import re


newline = lambda x: re.sub('\n', ' ', str(x)) # remove \n
contra_barra_erre =  lambda x: re.sub(r'[^ \w\.]', '', x).lower()
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x) # remove alphanumeric words

df_empresa['mda'] = df_empresa.mda.map(newline).map(contra_barra_erre).map(alphanumeric)
df_empresa['mda'][0]

'.    the following discussion should be read in conjunction with the consolidated financial statements as of  december      and  december      and for each of the three years in the period ended  december      and related notes which are included in this annual report on form   as well as with the other sections of this annual report on form   including part i item   business part ii item   selected financial data and part ii item   financial statements and supplementary data.  introduction  in this section we will describe the general financial condition and the results of operations of advanced micro devices inc. and its whollyowned subsidiaries collectively us our or amd including a discussion of our results of operations for     compared to      an analysis of changes in our financial condition and a discussion of our contractual obligations and offbalance sheet arrangements. discussions of   items and yeartoyear comparisons between   and   that are not included in this form   can

##Solução Tokenizer

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df_empresa['mda_token']=df_empresa.mda.map(sent_tokenize)
df_empresa

,mda,Symbol,mda_token
0,. the following discussion should be read i...,AMD,"[., the following discussion should be read in..."
1,this combined relates to the consolidat...,ED,[ this combined relates to the consolida...
2,. for discussion of results yearoveryear ...,GLW,"[., for discussion of results yearoveryear c..."
3,. mda is designed to provide a reader of d...,DHR,"[., mda is designed to provide a reader of dan..."
4,. this discussion and analysis below for da...,DRI,"[., this discussion and analysis below for dar..."
...,...,...,...
500,. forwardlooking statements statements in ...,PTC,"[., forwardlooking statements statements in t..."
501,. the following discussion should be read...,CRL,"[., the following discussion should be read in..."
502,nan,OGN,[nan]
503,. you should read the following discussion a...,MRNA,"[., you should read the following discussion a..."


In [ ]:
df_empresa.to_pickle('/content/drive/MyDrive/TCC/Bases Token/df_empresa.pkl')


#Loop

In [4]:
!pip install transformers

     |████████████████████████████████| 2.9 MB 4.1 MB/s 
     |████████████████████████████████| 56 kB 3.9 MB/s 
     |████████████████████████████████| 3.3 MB 53.7 MB/s 
     |████████████████████████████████| 636 kB 50.2 MB/s 
     |████████████████████████████████| 895 kB 55.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import pickle
import pandas as pd

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

lista_sentimento=[]


for c in range(0,505):
  with open('/content/drive/MyDrive/TCC/Bases Token/df_empresa_2017.pkl', 'rb') as f:  # jupyter notebook saved
    df_empresa = pickle.load(f)
  f.close()
  x=df_empresa.mda_token[c]
  print(df_empresa.Symbol[c],'',c,'/',505)
  df_empresa=[]
  zeta = [i for i in x if len(i)>30 and len(i)<503]
  x=[]
  lista=[]
  sentimento=0

  for j in list(range(0,len(zeta),50)):
    sentences=[]
    inputs=[]
    outputs=[]
    sentences = zeta[j:j+50]
    print(j)
    inputs = tokenizer(sentences, return_tensors="pt", padding=True)
    outputs = finbert(**inputs)[0]
    labels = {0:0, 1:1,2:-1}
    
    for idx, sent in enumerate(sentences):
        indice = np.argmax(outputs.detach().numpy()[idx])
        direcao = labels[indice]
        nota = outputs.detach().numpy()[idx][indice]
        lista.append(nota*direcao)

    teta = [i for i in lista if i!=0]
    if len(teta)!=0:
      sentimento=sum(teta)/len(teta)
    else:
      sentimento=0
  if c==0:
    with open('/content/drive/MyDrive/TCC/Bases_pt_2/mda_2017.pkl', 'rb') as f:
      df = pickle.load(f)
    f.close()
  else:
    with open('/content/drive/MyDrive/TCC/Bases/mda_result.pkl', 'rb') as f:
      df = pickle.load(f)
    f.close()
  df.loc[c, 'sentimento'] = sentimento
  df.to_pickle('/content/drive/MyDrive/TCC/Bases/mda_result.pkl')
  df=[]

AMD  0 / 505
0
50
100
150
200
250
300
350
400
450
ED  1 / 505
0
50
100
150
200
250
300
350
400
450
500
GLW  2 / 505
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
DHR  3 / 505
0
50
100
150
200
250
300
350
400
450
500
550
DRI  4 / 505
0
50
100
150
200
250
300
350
D  5 / 505
0
50
100
150
200
250
300
350
400
450
500
550
600
ETN  6 / 505
0
ES  7 / 505
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
FITB  8 / 505
FE  9 / 505
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
BEN  10 / 505
0
50
100
150
200
250
300
350
400
450
500
550
FCX  11 / 505
GE  12 / 505
HUM  13 / 505
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
HBAN  14 / 505
0
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
KLAC  15 / 505
0
50
100
150
200
250
300
350
400
450
L  16 / 505
0
50
100
150
200
250
300
350
400
450
500
550
6

In [12]:
import pickle
with open('/content/drive/MyDrive/TCC/Base Final/mda__2019.pkl', 'rb') as f:
      df = pickle.load(f)
f.close()

In [14]:
df[df.sentimento==df.sentimento.max()]

,Unnamed: 0,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Form Type,company_name,Date Filed,File Name,Url,mda_2020_1,sentimento
494,494,TRMB,Trimble,reports,Information Technology,Electronic Equipment & Instruments,"Sunnyvale, California",2021-01-21,864749,1978,10-K,TRIMBLE INC.,2019-02-22,864749_0000864749-19-000006.mda,https://www.sec.gov/Archives/edgar/data/864749...,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYS...,6.12819


In [1]:
import pickle
with open('/content/drive/MyDrive/TCC/Bases/mda_result.pkl', 'rb') as f:
      dp = pickle.load(f)
f.close()
dp

,Unnamed: 0,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded,Form Type,company_name,Date Filed,File Name,Url,mda_2020_1,sentimento
0,0,AMD,Advanced Micro Devices,reports,Information Technology,Semiconductors,"Santa Clara, California",NaN,2488,1969,10-K,ADVANCED MICRO DEVICES INC,2018-02-27,2488_0000002488-18-000042.mda,https://www.sec.gov/Archives/edgar/data/2488/0...,ITEM 7.\n\nMANAGEMENT’S DISCUSSION AND ANALYSI...,1.63866
1,1,ED,Consolidated Edison,reports,Utilities,Electric Utilities,"New York City, New York",NaN,1047862,1823,10-K,CONSOLIDATED EDISON INC,2018-02-15,1047862_0001047862-18-000037.mda,https://www.sec.gov/Archives/edgar/data/104786...,ITEM 7: MANAGEMENT’S DISCUSSION AND ANALYSI...,0.153854
2,2,GLW,Corning,reports,Information Technology,Electronic Components,"Corning, New York",NaN,24741,1851,10-K,CORNING INC /NY,2018-02-15,24741_0000024741-18-000010.mda,https://www.sec.gov/Archives/edgar/data/24741/...,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS ...,2.29216
3,3,DHR,Danaher Corporation,reports,Health Care,Health Care Equipment,"Washington, D.C.",NaN,313616,1969,10-K,DANAHER CORP /DE/,2018-02-21,313616_0000313616-18-000038.mda,https://www.sec.gov/Archives/edgar/data/313616...,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS ...,4.2074
4,4,DRI,Darden Restaurants,reports,Consumer Discretionary,Restaurants,"Orlando, Florida",NaN,940944,1938,10-K,DARDEN RESTAURANTS INC,2018-07-20,940944_0000940944-18-000050.mda,https://www.sec.gov/Archives/edgar/data/940944...,ITEM 7.\n\nMANAGEMENT’S DISCUSSION AND ANALYSI...,2.42699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,500,PTC,PTC,reports,Information Technology,Application Software,"Boston, Massachusetts",2021-04-20,857005,1985,10-K,PTC INC.,2018-11-16,857005_0000857005-18-000025.mda,https://www.sec.gov/Archives/edgar/data/857005...,ITEM 7.\n\nMANAGEMENT’S DISCUSSION AND ANALYSI...,3.36006
501,501,CRL,Charles River Laboratories,reports,Health Care,Life Sciences Tools & Services,"Wilmington, Massachusetts",2021-05-14,1100682,1947,10-K,CHARLES RIVER LABORATORIES INTERNATIONAL INC,2018-02-13,1100682_0001100682-18-000005.mda,https://www.sec.gov/Archives/edgar/data/110068...,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSI...,3.66592
502,502,OGN,Organon & Co.,reports,Health Care,Pharmaceuticals,"Jersey City, New Jersey",2021-06-03,1821825,2021,NaN,NaN,NaN,nan,NaN,NaN,0
503,503,MRNA,Moderna,reports,Health Care,Biotechnology,"Cambridge, Massachusetts",2021-07-21,1682852,2010,NaN,NaN,NaN,nan,NaN,NaN,0
